In [1]:
import os
import copy
import pickle
import sympy
import functools
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from error_injection import MissingValueError, SamplingError, Injector
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import mutual_info_score, auc, roc_curve, roc_auc_score, f1_score
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize as scipy_min
from scipy.spatial import ConvexHull
from scipy.optimize import minimize, Bounds, linprog
from sympy import Symbol as sb
from sympy import lambdify
from tqdm.notebook import trange,tqdm
from IPython.display import display,clear_output
from random import choice

import cProfile
import time
import pstats

import yep
import sys

np.random.seed(1)

In [2]:
profiler = cProfile.Profile()

In [3]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
def create_symbol(suffix=''):
    global symbol_id
    symbol_id += 1
    name = f'e{symbol_id}_{suffix}' if suffix else f'e{symbol_id}'
    return sympy.Symbol(name=name)

In [5]:
# helper functions from causal inference not appeared in zono_reg library

symbol_id = -1
scaler_symbols = set([sb(f'k{i}') for i in range(100)])
linearization_dict = dict()
reverse_linearization_dict = dict()


def compute_closed_form(X, y):
    return np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))

# def sample_data(imputed_datasets, uncert_inds=[], seed=42):
#     imp_np = np.array(imputed_datasets)
#     if len(uncert_inds) == 0:
#         uncert_inds = list(itertools.product(range(imp_np.shape[1]),range(imp_np.shape[2])))
#     np.random.seed(seed)
#     choices = np.random.choice(np.arange(imp_np.shape[0]), len(uncert_inds), replace=True)
#     sample_result = imputed_datasets[0].copy()
#     for i, ind in enumerate(uncert_inds):
#         sample_result[ind[0]][ind[1]] = imputed_datasets[choices[i]][ind[0]][ind[1]]
#     return sample_result

def sample_data(uncert_inds, uncertain_attr, X_extended_max, X_extended_min, seed=42):
    np.random.seed(seed)
    sample_result = []
    for u in uncert_inds:
        maximum = X_extended_max[u, uncertain_attr]
        minimum = X_extended_min[u, uncertain_attr]
        sample_result.append(np.random.uniform(minimum, maximum))
    return sample_result

# X_extended_max[:, uncertain_attr]


def linearization(expr_ls):
    # processed_expr_ls = [0 for _ in range(len(expr_ls))]
    processed_expr_ls = []
    print(len(expr_ls))
    for expr_id, expr in enumerate(expr_ls):
        print("Linearization", expr_id)
        # Do not support monomial expr currently, e.g., expr = 1.5*e1. 
        # At lease two monomials in expr, e.g., expr = 1.5*e1 + 2.
        if not(expr.free_symbols):
            processed_expr_ls[expr_id] += expr
            continue
        expr = expr.expand()
        processed_expr_ls.append(expr)
        # for arg in expr.args:
        #     print(arg)
        #     if not(arg.free_symbols):
        #         processed_expr_ls[expr_id] += arg
        #         continue
        #     p = arg.as_poly()
        #     monomial_exponents = p.monoms()[0]
            
        #     # only deal with non-linear monomials (order > 2)
        #     if sum(monomial_exponents) <= 1:
        #         processed_expr_ls[expr_id] += arg
        #         continue

        #     monomial = sympy.prod(x**k for x, k in zip(p.gens, monomial_exponents) 
        #                           if not(x in scaler_symbols))
        #     # check global substitution dictionary
        #     if monomial in linearization_dict:
        #         processed_expr_ls[expr_id] += arg.coeff(monomial)*linearization_dict[monomial]
        #     else:
        #         print("not in dict")
        #         found = False
        #         subs_monomial = create_symbol()
        #         for symb in monomial.free_symbols:
        #             if symb in reverse_linearization_dict:
        #                 equivalent_monomial = monomial.subs(symb, reverse_linearization_dict[symb])
        #                 if equivalent_monomial in linearization_dict:
        #                     subs_monomial = linearization_dict[equivalent_monomial]
        #                     found = True
        #                     break
        #         linearization_dict[monomial] = subs_monomial
        #         if not(found):
        #             reverse_linearization_dict[subs_monomial] = monomial
        #         processed_expr_ls[expr_id] += arg.coeff(monomial)*subs_monomial
                
    return processed_expr_ls


def merge_small_components_pca(expr_ls, budget=10):
    if not(isinstance(expr_ls, sympy.Expr)):
        expr_ls = sympy.Matrix(expr_ls)
    if expr_ls.free_symbols:
        center = expr_ls.subs(dict([(symb, 0) for symb in expr_ls.free_symbols]))
    else:
        return expr_ls
    monomials_dict = get_generators(expr_ls)
    generators = np.array([monomials_dict[m] for m in monomials_dict])
    if len(generators) <= budget:
        return expr_ls
    monomials = [m for m in monomials_dict]
    pca = PCA(n_components=len(generators[0]))
    pca.fit(np.concatenate([generators, -generators]))
    transformed_generators = pca.transform(generators)
    transformed_generator_norms = np.linalg.norm(transformed_generators, axis=1, ord=2)
    # from largest to lowest norm
    sorted_indices = transformed_generator_norms.argsort()[::-1].astype(int)
    sorted_transformed_generators = transformed_generators[sorted_indices]
    sorted_monomials = [monomials[idx] for idx in sorted_indices]
    new_transformed_generators = np.concatenate([sorted_transformed_generators[:budget], 
                                                 np.diag(np.sum(np.abs(sorted_transformed_generators[budget:]), 
                                                                axis=0))])
    new_generators = pca.inverse_transform(new_transformed_generators)
    new_monomials = sorted_monomials[:budget] + [create_symbol() for _ in range(len(generators[0]))]
    
    processed_expr_ls = center
    for monomial_id in range(len(new_monomials)):
        processed_expr_ls += sympy.Matrix(new_generators[monomial_id])*new_monomials[monomial_id]
    
    return processed_expr_ls


def get_vertices(affset):
    l = len(affset)
    distinct_symbols = set()
    for expr in affset:
        if not(isinstance(expr, sympy.Expr)):
            assert isinstance(expr, int) or isinstance(expr, float)
        else:
            if distinct_symbols:
                distinct_symbols = distinct_symbols.union(expr.free_symbols)
            else:
                distinct_symbols = expr.free_symbols
    distinct_symbols = list(distinct_symbols)
    # print(distinct_symbols)
    combs = [list(zip(distinct_symbols,list(l))) for l in list(itertools.product([-1, 1], repeat=len(distinct_symbols)))]
    res = set()
    for assignment in combs:
        res.add(tuple([expr.subs(assignment) for expr in affset]))
    return(res)

In [6]:
def zorro(ss, X_train, y_train, X_test, y_test, robustness_radius, uncertain_attr, 
                                 uncertain_num, uncertain_radius=None, uncertain_radius_ratio=None, 
                                 lr=0.03, reg=0, seed=42):
    start_time = time.perf_counter()
    X = copy.deepcopy(X_train)
    y = copy.deepcopy(y_train)
    
    symbol_id = -1
    symbolic_data = X.tolist()
    symbols_in_data = set()
    symbol_to_position = dict()
    XS = X.tolist()
    XR = X.tolist()
    for row in range(len(symbolic_data)):
        for col in range(len(symbolic_data[0])):
            xmin = X_extended_min[row][col]
            xmax = X_extended_max[row][col]
            if xmin != xmax:
                xmean = (xmax + xmin) / 2
                xradius = (xmax - xmin) / 2
                new_symbol = create_symbol()
                symbolic_data[row][col] = xmean + xradius*new_symbol
                XS[row][col] = xradius*new_symbol
                XR[row][col] = xmean
                symbols_in_data.add(new_symbol)
                symbol_to_position[new_symbol] = (row, col)
            else:
                XS[row][col] = 0
                
    XS = sympy.Matrix(XS)
    XR = sympy.Matrix(XR)
    n = XS.shape[0]
    n1 = XS.shape[1]
    yR = sympy.Matrix(y_train)
    yS = yR*0.0
    y_test = sympy.Matrix(y_test.to_numpy().reshape(-1, 1))
    # X_test = sympy.Matrix(np.append(np.ones((len(X_test), 1)), X_test, axis=1))
    X_test = sympy.Matrix(np.append(np.ones((len(X_test), 1)), ss.transform(X_test), axis=1))

    # for row in range(n):
    #     for col in range(n1):
    #         expr = copy.deepcopy(XR[row, col])
    #         if isinstance(expr, sympy.Expr) and expr.free_symbols:
    #             XR[row, col] = expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))
    #             XS[row, col] = expr - XR[row, col]
    #         else:
    #             XR[row, col] = expr
    #             XS[row, col] = 0

    # for row in range(yR.shape[0]):
    #     expr = copy.deepcopy(yR[row])
    #     if isinstance(expr, sympy.Expr) and expr.free_symbols:
    #         yR[row] = expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))
    #         yS[row] = expr - yR[row]
    #     else:
    #         yR[row] = expr
    #         yS[row] = 0
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Data loading execution time: {execution_time} seconds")

    start_time = time.perf_counter()
    common_inv = (XR.T*XR + reg*n*np.identity(X.shape[1])).inv()
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Common inverse matrix computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    V, sigma, VT = np.linalg.svd(np.array((XR.T*XR).tolist()).astype(float))
    V = sympy.Matrix(V)
    VT = sympy.Matrix(VT)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"SVD computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    wR = common_inv*XR.T*yR
    wS_data = common_inv*((XS.T*XR + XR.T*XS)*wR - XS.T*yR - XR.T*yS)
    wS_non_data = 0.0*VT.row(0).T
    for i in range(X.shape[1]):
        wS_non_data = wS_non_data + sb(f'k{i}')*sb(f'ep{i}')*VT.row(i).T
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Abstract weights computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    eigenvalues = 1 - 2*lr*reg - 2*lr*sigma/n
    for eigenvalue in eigenvalues:
        assert eigenvalue <= 1
        assert eigenvalue >= 0
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Eigenvalue computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    wS = wS_non_data + wS_data
    w = wS + wR
    w_prime = (-lr*2/n)*((XS.T*XR + XR.T*XS + XS.T*XS)*wS + XS.T*XS*wR - XS.T*yS).expand()
    coeff_cont = (-lr*2/n)*((XS.T*XR + XR.T*XS + XS.T*XS)*wS_non_data).expand()
    w_prime_projected = VT*w_prime
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"W# (abstract gradient descent?) computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    eqs = []
    for d in tqdm(range(X.shape[1]), desc='Equation'):
        eq1 = (1-abs(eigenvalues[d]))*sb(f'k{d}')
        eq2 = 0
        coef_dict = dict()
        coef_dict['const'] = 0
        for i in range(X.shape[1]):
            coef_dict[sb(f'k{i}')] = 0
        for arg in w_prime_projected[d].args:
            contain_k = False
            for i in range(X.shape[1]):
                symb_k = sb(f'k{i}')
                if symb_k in arg.free_symbols:
                    coef_dict[symb_k] = coef_dict[symb_k] + abs(arg.args[0])
                    contain_k = True
                    break
            if not(contain_k):
                coef_dict['const'] = coef_dict['const'] + abs(arg.args[0])
        eq2 = coef_dict['const']
        for i in range(X.shape[1]):
            eq2 = eq2 + sb(f'k{i}')*coef_dict[sb(f'k{i}')]
        eqs.append(sympy.Eq(eq1, eq2))
        
    result = sympy.solve(eqs, [sb(f'k{i}') for i in range(X.shape[1])])
    print(result)
    for ki in result:
        assert result[ki] >= 0
    param = wR + wS.subs(result)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Equation solving time: {execution_time} seconds")
    
    # center_preds = X_test*get_expr_center(param)
    # print('one-poss-world:', [((center_preds - y_test).T*(center_preds - y_test)/n)[0]])
    start_time = time.perf_counter()
    test_preds = X_test*param
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Prediction time: {execution_time} seconds")
    print(test_preds[0])
    # robustness_ls = []
#     se_min_ls = []
#     se_max_ls = []
    # acc_count_low = 0
    # acc_count_high = 0
    # for pred_id in tqdm(range(len(test_preds)), desc='Testing'):
    #     pred = test_preds[pred_id]
    #     label_test = y_test[pred_id]
    #     pred_range_radius = get_expr_range_radius(pred)
    #     if pred_range_radius <= robustness_radius:
    #         robustness_ls.append(1)
    #     else:
    #         robustness_ls.append(0)
    #     pred_center = get_expr_center(pred)
    #     pred_low = pred_center - pred_range_radius
    #     pred_hi = pred_center + pred_range_radius
    #     # get accuracy lower and upper bound
    #     if pred_hi < 0.5 and label_test == 0:
    #         acc_count_low += 1
    #         acc_count_high += 1
    #     elif pred_low > 0.5 and label_test == 1:
    #         acc_count_low += 1
    #         acc_count_high += 1
    #     elif pred_low < 0.5 and pred_hi > 0.5:
    #         acc_count_high += 1
#         rewrite_pred = pred_center+pred_range_radius*sb('err_pred')
#         se_min, se_max = min_max_sympy_expression((rewrite_pred-label_test)**2)
#         se_min_ls.append(se_min)
#         se_max_ls.append(se_max)
    # preds_diff = (test_preds - y_test).T*(test_preds - y_test)/n
    start_time = time.perf_counter()
    n = X_test.shape[0]
    preds_diff = ((test_preds - y_test).T*(test_preds - y_test)/n)[0]
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Loss computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    preds_diff_full = linearization([preds_diff])[0]
    # preds_diff_center = get_expr_center(preds_diff)
    # preds_diff_radius = get_expr_range_radius(preds_diff)
#     print(preds_diff_center)
#     print(preds_diff_radius)
    # print("Robustness Ratio: " + str(np.mean(robustness_ls)))
    
#     print(param)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Loss linearization time: {execution_time} seconds")
    return result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff, preds_diff_full, X_test

In [7]:
def get_expr_range_radius(expr):
    expr_range_radius = 0
    for arg in expr.args:
        if arg.free_symbols:
            expr_range_radius += abs(arg.args[0])
    return expr_range_radius

def get_expr_center(expr):
    return expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))

In [8]:
X_train = pd.read_csv('dataset/Insurance_injector_lr/real/X_train_clean.csv')
X_train_dirty = pd.read_csv('dataset/Insurance_injector_lr/real/X_train_dirty.csv')

In [9]:
injected_size = 127

In [10]:
y_train = pd.read_csv('dataset/Insurance_injector_lr/real/y_train.csv')
y_train.shape[0]

802

In [11]:
y_train = pd.DataFrame(y_train, columns=['charges'])
y_train

,charges
0,0
1,1
2,1
3,1
4,1
...,...
797,1
798,0
799,1
800,0


In [12]:
df = X_train.merge(y_train, left_index=True, right_index=True)
def compute_closed_form(X, y):
    return np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))

In [13]:
dirty_df = X_train_dirty.merge(y_train, left_index=True, right_index=True)

In [14]:
all_cols = ['age', 'bmi', 'children']
all_cols_idx = [df.columns.to_list().index(c) for c in all_cols]

# num of errors injected
# mv_num = 30

# which col to inject missing val
treatment_idx = all_cols.index('bmi')
uncertain_attr = all_cols.index('children')
# uncertain_attr = [all_cols.index('displacement'), all_cols.index('weight')]

# MNAR
# def non_random_pattern(data_X, data_y):
#     binary_indicators = []
#     for i in range(data_X.shape[0]):
#         if (data_X.iloc[i, treatment_idx] > 25) and (data_X.iloc[i, uncertain_attr] > 0):
#             binary_indicators.append(1)
#         else:
#             binary_indicators.append(0)
#     return np.array(binary_indicators)

# sample_pattern_len = np.sum(non_random_pattern(df.drop('cylinders', axis=1).copy(), df.cylinders))
# mv_ratio = mv_num/sample_pattern_len
# print(mv_ratio)
# mv_err = MissingValueError(uncertain_attr, pattern=non_random_pattern, ratio=mv_ratio)
# dirty_df, dirty_y, _, _ = mv_err.inject(df.drop('cylinders', axis=1).copy(), df.cylinders, df.drop('cylinders', axis=1), df.cylinders)

X_extended = np.append(np.ones((len(dirty_df), 1)), 
                       dirty_df.to_numpy().astype(float)[:, all_cols_idx], axis=1)
X_extended_clean = np.append(np.ones((len(df), 1)), 
                             df.to_numpy().astype(float)[:, all_cols_idx], axis=1)

ss = StandardScaler()
X_extended[:, 1:] = ss.fit_transform(X_extended[:, 1:])
X_extended_clean[:, 1:] = ss.transform(X_extended_clean[:, 1:])

param_clean = compute_closed_form(X_extended_clean, y_train)

# first column becomes constant 1's
treatment_idx += 1
uncertain_attr += 1
# remain_col_idx = [0] + [i+1 for i in remain_col_idx]

In [15]:
imputers = [KNNImputer(n_neighbors=3), KNNImputer(n_neighbors=5), KNNImputer(n_neighbors=10), IterativeImputer(random_state=42)]
num_attrs = X_extended.shape[1]
X_nan = X_extended.copy()
imputed_cols = [X_extended_clean[:, uncertain_attr]]
# imputed_other = [X_extended_clean[:, other_uncertain]]
imputed_datasets = [X_extended_clean]
for imp in imputers:
    imputed_dataset = imp.fit_transform(X_nan)
    imputed_datasets.append(imputed_dataset)
    imputed_cols.append(imputed_dataset[:, uncertain_attr])
    # imputed_other.append(imputed_dataset[:, other_uncertain])

X_extended_max = X_extended.copy()
X_extended_max[:, uncertain_attr] = np.max(imputed_cols, axis=0)
# X_extended_max[:, other_uncertain] = np.max(imputed_other, axis=0)
# X_extended_max = X_extended_max[:, remain_col_idx]

X_extended_min = X_extended.copy()
X_extended_min[:, uncertain_attr] = np.min(imputed_cols, axis=0)
# X_extended_min[:, other_uncertain] = np.min(imputed_other, axis=0)
# X_extended_min = X_extended_min[:, remain_col_idx]

# X_extended = X_extended[:, remain_col_idx]

In [16]:
X_test = pd.read_csv('dataset/Insurance_injector_lr/real/X_test.csv')
y_test = pd.read_csv('dataset/Insurance_injector_lr/real/y_test.csv')
dirty_y = y_train

In [17]:
X_val = pd.read_csv('dataset/Insurance_injector_lr/real/X_val.csv')
y_val = pd.read_csv('dataset/Insurance_injector_lr/real/y_val.csv')

In [18]:
df_diff = pd.concat([X_train, X_train_dirty])
df_diff = df_diff.drop_duplicates(keep=False)

In [19]:
injected_indices = []
for i in range(len(dirty_df)):
    if pd.isna(dirty_df.iloc[i]['children']):
        injected_indices.append(i)

injected_indices = np.array(injected_indices)
injected_indices

array([  1,   2,   9,  13,  20,  22,  23,  29,  30,  48,  52,  57,  68,
        71,  73,  91,  93,  96, 104, 105, 107, 108, 118, 128, 129, 130,
       134, 143, 151, 152, 155, 156, 169, 194, 199, 204, 225, 228, 236,
       237, 238, 241, 247, 254, 274, 277, 306, 315, 318, 320, 339, 346,
       352, 354, 361, 365, 366, 367, 368, 375, 378, 383, 384, 391, 393,
       407, 411, 422, 423, 427, 429, 434, 437, 445, 447, 456, 457, 471,
       493, 496, 506, 519, 520, 529, 531, 536, 540, 543, 562, 568, 581,
       585, 587, 588, 592, 598, 615, 632, 636, 638, 656, 657, 663, 672,
       676, 679, 684, 696, 698, 710, 713, 714, 718, 721, 737, 738, 743,
       754, 756, 762, 764, 768, 775, 784, 790, 797, 799])

In [20]:
len(injected_indices)

127

In [24]:
profiler.enable()
result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff, preds_diff_full, X_test_matrix = zorro(ss, X_extended, dirty_y, X_val, y_val, 0.05, uncertain_attr, 
                             injected_size, uncertain_radius=None, uncertain_radius_ratio=None, lr=0.01, reg=0, seed=42)

Data loading execution time: 0.2702076590003344 seconds
Common inverse matrix computation time: 0.2791182460005075 seconds
SVD computation time: 0.1489198130002478 seconds
Abstract weights computation time: 2.984412921000512 seconds
Eigenvalue computation time: 4.560900015349034e-05 seconds
W# (abstract gradient descent?) computation time: 172.05908391700086 seconds


Equation:   0%|          | 0/4 [00:00<?, ?it/s]

{k0: 0.0266298794545128, k1: 0.0622455095121689, k2: 0.0219989802986302, k3: 0.0739433927527262}
Equation solving time: 4.725314878999598 seconds
Prediction time: 49.68225709500075 seconds
0.000947039963634905*e0 + 0.000847804913764051*e1 + 0.000368434422772985*e10 + 0.000314783319687061*e100 + 0.00125243278096149*e101 + 0.000347370265345479*e102 + 0.000172265790517063*e103 + 0.00126552074470456*e104 + 0.000333791804029507*e105 + 0.000632631685800814*e106 + 0.000328430770640383*e107 + 0.000453950234858534*e108 + 0.00084553140372874*e109 + 0.000809103197656317*e11 + 0.000322539711527024*e110 + 0.00043904058734371*e111 + 0.000850001158617883*e112 + 0.000180363683265759*e113 + 0.000132228131414142*e114 + 9.94795703289388e-5*e115 - 1.77176265822585e-5*e116 + 0.000688534316600768*e117 + 0.000853509235474776*e118 + 9.67577615063122e-5*e119 + 0.000399829725309448*e12 + 0.00134599951859184*e120 + 0.00025270683794827*e121 + 0.000146463367973991*e122 + 0.00139009691339604*e123 + 0.00043750973271

In [25]:
profiler.disable()
stats2 = pstats.Stats(profiler)
stats2.sort_stats('cumulative').print_stats(100)

         5428594980 function calls (5228475923 primitive calls) in 1671.038 seconds

   Ordered by: cumulative time
   List reduced from 2213 to 100 due to restriction <100>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000 1671.029  835.514 /home/hky/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3541(run_code)
     25/2    0.000    0.000 1671.029  835.514 {built-in method builtins.exec}
        1    0.039    0.039 1671.029 1671.029 /tmp/ipykernel_3913/919804467.py:1(<module>)
        1    0.410    0.410 1670.990 1670.990 /tmp/ipykernel_3913/2968385090.py:1(zorro)
38685488/423780   33.938    0.000 1665.643    0.004 /home/hky/.local/lib/python3.10/site-packages/sympy/core/cache.py:69(wrapper)
6008386/2800961   42.574    0.000 1605.544    0.001 /home/hky/.local/lib/python3.10/site-packages/sympy/core/operations.py:52(__new__)
       17    0.003    0.000 1451.092   85.358 /home/hky/.local/lib/python3.10/site-package

In [54]:
with open('dataset/Insurance_injector_lr/ins_iter_model.pkl', 'wb') as file:
    pickle.dump(param, file)

In [55]:
with open('dataset/Insurance_injector_lr/X_test_matrix.pkl', 'wb') as file:
    pickle.dump(X_test_matrix, file)

In [56]:
with open('dataset/Insurance_injector_lr/y_test_ins.pkl', 'wb') as file:
    pickle.dump(y_test, file)

In [58]:
with open('dataset/Insurance_injector_lr/ins_iter_full.pkl', 'wb') as file:
    pickle.dump(preds_diff_full, file)

with open('dataset/Insurance_injector_lr/ins_iter.pkl', 'wb') as file:
    pickle.dump(preds_diff, file)

In [21]:
with open('dataset/Insurance_injector_lr/ins_iter_model.pkl', 'rb') as file:
    param = pickle.load(file)

In [22]:
with open('dataset/Insurance_injector_lr/X_test_matrix.pkl', 'rb') as file:
    X_test_matrix = pickle.load(file)

In [23]:
with open('dataset/Insurance_injector_lr/y_test_ins.pkl', 'rb') as file:
    y_test = pickle.load(file)

In [24]:
def sympy_to_ginac_format(expr):
    """Convert SymPy expression to GiNaC-parseable string format."""
    # Replace SymPy-specific functions with GiNaC equivalents
    s = str(expr)
    s = s.replace('**', '^')  # Exponentiation
    s = s.replace('sin', 'GiNaC::sin')
    s = s.replace('cos', 'GiNaC::cos')
    # Add more replacements as needed
    return s

In [25]:
def parse_ginac_to_sympy(ginac_expr_str, sympy_expr):
    """
    Parse a GiNaC expression string back into a SymPy expression
    """
    # Define SymPy symbols
    symbols = sympy_expr.free_symbols
    symbols_dict = dict()
    for s in symbols:
        symbols_dict[str(s)] = s
    
    # This parsing would need to be made more robust for real-world use
    # For now, a simple example to show the concept
    expr = ginac_expr_str
    for symbol_name, symbol_obj in symbols_dict.items():
        expr = expr.replace(symbol_name, f'symbols["{symbol_name}"]')
    
    # Very unsafe, but for demonstration only:
    return eval(expr, {"symbols": symbols_dict, "sympy": sympy})

In [26]:
import ginac_module

In [27]:
ginac_param = sympy_to_ginac_format(param)
X_test_list = list(np.array(X_test_matrix, dtype=float))
X_test_list = [list(l) for l in X_test_list]
y_test_list = [float(l[0]) for l in y_test.to_numpy()]

In [28]:
ginac_param_list = [sympy_to_ginac_format(s) for s in param]

In [29]:
profile_output = 'cpp_profiling_ins.prof'

In [30]:
profiler.clear()
profiler.enable()
start_time = time.perf_counter()
print(f"Starting C++ profiler, output will be in {profile_output}")
yep.start(profile_output)
result_strings = ginac_module.abstract_loss(X_test_list, y_test_list, ginac_param_list)
yep.stop()
print("C++ Profiling completed")
end_time = time.perf_counter()
profiler.disable()
execution_time = end_time - start_time
print(f"GiNaC time: {execution_time} seconds")

Starting C++ profiler, output will be in cpp_profiling_ins.prof
X: 268*4
y: 268*1
param: 4*1
Time for multiplication: 27 ms
Time for difference: 3 ms
Time for square: 13 ms
Time for expansion: 7 ms
Number of nodes in sum_squared: 422369
Number of terms in sum_squared: 268
Depth of sum_squared: 4
Current memory usage: 	  342492 kB
Expression has 268 terms at top level
Approximately 100% of terms have complex structure
Detected complex term structure, adjusting algorithm
Using optimized GiNaC operations for complex structure...
Time for combining like terms: 82644 ms
Number of nodes in sum_squared after expansion: 69074
Number of terms in sum_squared after expansion: 17269
Depth of sum_squared after expansion: 3
Current memory usage: 	 9007568 kB
C++ Profiling completed
GiNaC time: 81.54129200699981 seconds


PROFILE: interrupts/evictions/bytes = 8179/1081/1461520


In [31]:
stats2 = pstats.Stats(profiler)
stats2.sort_stats('cumulative').print_stats(100)

         232 function calls in 81.541 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000   81.536   10.192 /home/hky/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        8    0.000    0.000   81.536   10.192 {built-in method builtins.exec}
        1    0.001    0.001   81.390   81.390 /tmp/ipykernel_865/783118916.py:1(<module>)
        1   81.390   81.390   81.390   81.390 {built-in method ginac_module.abstract_loss}
        1    0.140    0.140    0.140    0.140 /home/hky/.local/lib/python3.10/site-packages/yep.py:56(stop)
        8    0.000    0.000    0.005    0.001 /usr/lib/python3.10/codeop.py:117(__call__)
        8    0.005    0.001    0.005    0.001 {built-in method builtins.compile}
        1    0.005    0.005    0.005    0.005 /home/hky/.local/lib/python3.10/site-packages/yep.py:32(start)
        2    0.000    0.000    0.000    0.000 {built-in method

In [34]:
print("\nTo analyze the C++ profiler results, run these commands in a terminal:")
print(f"google-pprof --gif {sys.executable} {profile_output} > prof_ins.gif")


To analyze the C++ profiler results, run these commands in a terminal:
google-pprof --gif /usr/bin/python3 cpp_profiling_ins.prof > prof_ins.gif


In [33]:
start_time = time.perf_counter()
result_sympy = []
    
for expr_str in result_strings:
    # Parse the GiNaC output to SymPy
    # This would need a proper parser for real use
    expr = sympy.sympify(expr_str)
    # expr = sympy.expand(expr)
    # expr = sympy.collect(expr, list(symbols))
        
    result_sympy.append(expr)

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"SymPy conversion time: {execution_time} seconds")

print("Result from GiNaC:")
for expr in result_sympy:
    print(expr)

RecursionError: maximum recursion depth exceeded during compilation

In [ ]:
result_strings[0]